# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [52]:
import pandas as pd
        
df = pd.read_csv('../../../../Datasets/previsao_de_renda.csv', sep = ',')

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   index                  15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12466 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  mau                    15000 non-null  bool   
 15  re

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

#### 1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.

In [54]:
import statsmodels.api as sm
import numpy as np
import statsmodels.formula.api as smf
import patsy    

qualitative = df.select_dtypes(include='object').columns    
qualitative = qualitative.drop('data_ref')
print(qualitative)

Index(['sexo', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia'], dtype='object')


In [55]:
# mantenha sempre a categoria mais frequente como casela de referência

#achando a categoria mais frequente
most_frequent = df[qualitative].mode().iloc[0]
print(most_frequent)
    

sexo                         F
tipo_renda         Assalariado
educacao            Secundário
estado_civil            Casado
tipo_residencia           Casa
Name: 0, dtype: object


In [56]:

formula = 'np.log(renda) ~ ' + ' + '.join(
    [f"C({col}, Treatment(reference='{most_frequent[col]}'))" for col in qualitative]
)

# Criar os arrays de design
y, X = patsy.dmatrices(formula, df, return_type='dataframe')

In [57]:

model = sm.OLS(y, X)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.080
Model:                            OLS   Adj. R-squared:                  0.079
Method:                 Least Squares   F-statistic:                     72.24
Date:                Thu, 16 May 2024   Prob (F-statistic):          7.05e-254
Time:                        15:24:17   Log-Likelihood:                -17264.
No. Observations:               15000   AIC:                         3.457e+04
Df Residuals:                   14981   BIC:                         3.471e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------

#### 2. Remova a variável menos significante e analise:

In [58]:
qualitative = qualitative.drop('estado_civil')

formula = 'np.log(renda) ~ ' + ' + '.join(
    [f"C({col}, Treatment(reference='{most_frequent[col]}'))" for col in qualitative]
)

# Criar os arrays de design
y, X = patsy.dmatrices(formula, df, return_type='dataframe')
model = sm.OLS(y, X)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.079
Method:                 Least Squares   F-statistic:                     92.37
Date:                Thu, 16 May 2024   Prob (F-statistic):          2.72e-256
Time:                        15:24:17   Log-Likelihood:                -17268.
No. Observations:               15000   AIC:                         3.457e+04
Df Residuals:                   14985   BIC:                         3.468e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------

#### 3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor.

In [59]:
# agrupando dummies com menos de 5% de significância

# Identificar as variáveis dummies com p-valores maiores que 0.05
p_values = result.pvalues
high_p_values = p_values[p_values > 0.05]
high_p_dummies = high_p_values.index.tolist()

X_reduced = X.drop(columns=high_p_dummies)

# Ajustar o modelo novamente com as variáveis restantes
model_reduced = sm.OLS(y, X_reduced)
result_reduced = model_reduced.fit()
result_reduced.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.079
Method:                 Least Squares   F-statistic:                     117.5
Date:                Thu, 16 May 2024   Prob (F-statistic):          4.45e-259
Time:                        15:24:17   Log-Likelihood:                -17268.
No. Observations:               15000   AIC:                         3.456e+04
Df Residuals:                   14988   BIC:                         3.465e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=========================================================================================================================================
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                 7.9509      0.011    696.798      0.000       7.928       7.973
C(sexo, Treatment(reference='F'))[T.M]                                    0.3915      0.014     28.706      0.000       0.365       0.418
C(tipo_renda, Treatment(reference='Assalariado'))[T.Empresário]           0.1198      0.016      7.526      0.000       0.089       0.151
C(tipo_renda, Treatment(reference='Assalariado'))[T.Pensionista]         -0.1204      0.018     -6.704      0.000      -0.156      -0.085
C(tipo_renda, Treatment(reference='Assalariado'))[T.Servidor público]     0.2232      0.023      9.732      0.000       0.178       0.268
C(educacao, Treatment(reference='Secundário'))[T.Primário]               -0.1534      0.056     -2.743      0.006      -0.263      -0.044
C(educacao, Treatment(reference='Secundário'))[T.Pós graduação]           0.4170      0.176      2.371      0.018       0.072       0.762
C(educacao, Treatment(reference='Secundário'))[T.Superior completo]       0.0794      0.013      5.908      0.000       0.053       0.106
C(educacao, Treatment(reference='Secundário'))[T.Superior incompleto]    -0.0892      0.033     -2.744      0.006      -0.153      -0.025
C(tipo_residencia, Treatment(reference='Casa'))[T.Aluguel]               -0.1191      0.057     -2.090      0.037      -0.231      -0.007
C(tipo_residencia, Treatment(reference='Casa'))[T.Com os pais]           -0.2321      0.031     -7.430      0.000      -0.293      -0.171
C(tipo_residencia, Treatment(reference='Casa'))[T.Governamental]         -0.1048      0.037     -2.834      0.005      -0.177      -0.032
==============================================================================
Omnibus:                       37.215   Durbin-Watson:                   2.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               39.033
Skew:                           0.099   Prob(JB):                     3.34e-09
Kurtosis:                       3.152   Cond. No.                         32.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Conclusão

    Sim o modelo melhorou, o F-statistic aumentou e o AIC e BIC diminuiram enquanto o R-squared ajustado se manteve o mesmo. Embora o modelo em si é bem ruim.